### Carregar bibliotecas

In [1]:
import pandas as pd
from datetime import datetime
import random
import sklearn
from sklearn import tree, neighbors, ensemble, discriminant_analysis, neural_network
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA
import joblib
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

### Carregar dados

In [2]:
dados = pd.read_csv('dados/treino.csv', sep=';')

dados.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type_Red,type_White
0,7.3,0.48,0.32,2.1,0.062,31.0,54.0,0.99728,3.30,0.65,10.0,7,1,0
1,7.7,0.41,0.76,1.8,0.611,8.0,45.0,0.99680,3.06,1.26,9.4,5,1,0
2,7.9,0.35,0.46,3.6,0.078,15.0,37.0,0.99730,3.35,0.86,12.8,8,1,0
3,8.5,0.19,0.48,1.1,0.026,23.0,58.0,0.99184,2.90,0.50,10.5,6,0,1
4,9.1,0.28,0.46,9.0,0.114,3.0,9.0,0.99901,3.18,0.60,10.9,6,1,0


### Separar dados em atributos (X) e classe (y)

In [3]:
X = dados.drop('quality', axis=1)
y = dados.quality

### Lista com todas as técnicas que serão utilizadas

In [4]:
tecnicas_regressao = [
    ('RandomForestRegressor', sklearn.ensemble.forest.RandomForestRegressor()),
    ('ExtraTreesRegressor', sklearn.ensemble.forest.ExtraTreesRegressor()),
    ('BaggingRegressor', sklearn.ensemble.bagging.BaggingRegressor()),
    ('GradientBoostingRegressor', sklearn.ensemble.gradient_boosting.GradientBoostingRegressor()),
    ('AdaBoostRegressor', sklearn.ensemble.weight_boosting.AdaBoostRegressor()),
    ('HuberRegressor', sklearn.linear_model.huber.HuberRegressor()),
    ('LinearRegression', sklearn.linear_model.base.LinearRegression()),
    ('PassiveAggressiveRegressor', sklearn.linear_model.passive_aggressive.PassiveAggressiveRegressor()),
    ('SGDRegressor', sklearn.linear_model.stochastic_gradient.SGDRegressor()),
    ('TheilSenRegressor', sklearn.linear_model.theil_sen.TheilSenRegressor()),
    ('KNeighborsRegressor', sklearn.neighbors.regression.KNeighborsRegressor()),
    ('RadiusNeighborsRegressor', sklearn.neighbors.regression.RadiusNeighborsRegressor()),
    ('MLPRegressor', sklearn.neural_network.multilayer_perceptron.MLPRegressor()),
    ('DecisionTreeRegressor', sklearn.tree.tree.DecisionTreeRegressor()),
    ('ExtraTreeRegressor', sklearn.tree.tree.ExtraTreeRegressor())
]

len(tecnicas_regressao)

15

In [5]:
def log(mensagem):
    """
    Função para receber uma mensagem e exibir.
    Futuramente esta função pode receber a funcionalidade de salvar as mensagens de log em arquivo
    """
    print(str(datetime.now()) + ': ' + mensagem)

In [6]:
def getNumeroAleatorio(tipo, maximo=None):
    """
    Função para retornar um número aleatório de acordo com o tipo que é passado por parâmetro.        
    """
    
    if tipo == 'cross_validation': 
        return random.randint(3,10) # Definido que a quantidade de divisão dos dados (split) será entre 3 a 10 partes (folds)
    elif tipo == 'random_state':
        return random.randint(1, 42) # Valor do random state poderá variar entre 1 e 42
    elif tipo == 'simples':  # Retorna um número entre 0 e o valor passado por parâmetro
        return random.randint(0,maximo)

In [7]:
def splitDados(X, y):
    """
    Função para separar os dados de treino e teste, escolhendo aleatoriamente qual será a forma de separar os dados.
    Formas disponíveis de separação:
        - 80% para treino e 20% para teste
        - 75% para treino e 25% para teste
        - 70% para treino e 30% para teste
    """
    
    aleatorio = getNumeroAleatorio('simples', 2)
    
    random_state = getNumeroAleatorio('random_state')
    
    if aleatorio == 0:
        tipo_split = 'train_test_split 20%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    elif aleatorio == 1:
        tipo_split = 'train_test_split 25%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)
    elif aleatorio == 2:
        tipo_split = 'train_test_split 30%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    
    return X_train, X_test, y_train, y_test, tipo_split, random_state

In [8]:
def getNormalizador():
    """
    Retorna aleatoriamente qual normalizador será utilizado ou se não será utilizado normalizador.
    Se algum normalizador for escolhido, é retornado a lista de parâmetros do normalizador escolhido.
    Normalizadores disponíveis:
        - StandardScaler
        - RobustScaler
        - MinMaxScaler
        - Normalizer
    """
    
    normalizador = [(),
                    ('standard_scaler', StandardScaler()),
                    ('robust_scaler', RobustScaler()),
                    ('min_max_scaler', MinMaxScaler()),
                    ('normalizer', Normalizer())
                   ]
    
    aleatorio = getNumeroAleatorio('simples', len(normalizador)-1)
    
    if aleatorio == 0:
        parametros = {}
    elif normalizador[aleatorio][0] == 'standard_scaler':
        parametros = {
            'standard_scaler__with_mean': [True, False],
            'standard_scaler__with_std': [True, False]
        }
    elif normalizador[aleatorio][0] == 'robust_scaler':
        parametros = {
            'robust_scaler__with_centering': [True, False],
            'robust_scaler__with_scaling': [True, False]
        }
    elif normalizador[aleatorio][0] == 'min_max_scaler':
        parametros = {
            'min_max_scaler__feature_range': [(0,1), (1,10), (1,100)]
        }
    elif normalizador[aleatorio][0] == 'normalizer':
        parametros = {
            'normalizer__norm': ('l1', 'l2', 'max')
        }
        
    
    return normalizador[aleatorio], parametros

In [9]:
def getRedutorDimensionalidade():
    """
    Retorna aleatoriamente qual redutor de dimensionalidade será utilizado ou se não será utilizado redutor de dimensionalidade.
    Se for escolhido um redutor de dimensionalidade, é retornado a lista de parâmetros do redutor de dimensionalidade escolhido.
    Redutor de dimensionalidade disponível:
        - PCA
    """    
    
    redutor_dimensionalidade = [(),
                                ('pca', PCA())
                               ]
    
    aleatorio = getNumeroAleatorio('simples', len(redutor_dimensionalidade)-1)
    
    if aleatorio == 0:
        parametros = {}
    elif redutor_dimensionalidade[aleatorio][0] == 'pca':
        parametros = {
            'pca__n_components': [None, 3, 5, 7, 9, 11],
            'pca__whiten': [True, False],
            'pca__svd_solver': ('auto', 'full', 'randomized')
        }
        
    
    return redutor_dimensionalidade[aleatorio], parametros

In [10]:
def getParametrosTecnica(tecnica):
    """
    Recebe o nome de uma técnica de predição por parâmetro, e retorna os parâmetros da técnica
    """
    
    if  tecnica == 'LinearRegression':
        parametros = {
            'LinearRegression__fit_intercept': [True, False],
            'LinearRegression__normalize': [True, False]
        }
    elif tecnica == 'RandomForestRegressor':
        parametros = {
            'RandomForestRegressor__n_estimators': [5, 10, 15],
            'RandomForestRegressor__criterion': ('mse', 'mae'),
            'RandomForestRegressor__max_features': ('auto', 'sqrt', 'log2'),
            'RandomForestRegressor__bootstrap': [True, False],
            'RandomForestRegressor__warm_start': [True, False]
        }
    elif tecnica == 'ExtraTreesRegressor':
        parametros = {
            'ExtraTreesRegressor__n_estimators': [5, 10, 15],
            'ExtraTreesRegressor__criterion': ('mse', 'mae'),
            'ExtraTreesRegressor__max_features': ('auto', 'sqrt', 'log2'),
            'ExtraTreesRegressor__bootstrap': [True, False],
            'ExtraTreesRegressor__warm_start': [True, False]
        }
    elif tecnica == 'BaggingRegressor':
        parametros = {
            'BaggingRegressor__n_estimators': [5, 10, 15],
            'BaggingRegressor__bootstrap': [True, False],
            'BaggingRegressor__warm_start': [True, False]
        }
    elif tecnica == 'GradientBoostingRegressor':
        parametros = {
            'GradientBoostingRegressor__loss': ('ls', 'lad', 'huber', 'quantile'),
            'GradientBoostingRegressor__criterion': ('friedman_mse', 'mse', 'mae'),
            'GradientBoostingRegressor__max_features': ('auto', 'sqrt', 'log2'),
            'GradientBoostingRegressor__n_estimators': [50, 100, 150],
            'GradientBoostingRegressor__warm_start': [True, False]
        }
    elif tecnica == 'AdaBoostRegressor':
        parametros = {
            'AdaBoostRegressor__loss': ('linear', 'square', 'exponential'),
            'AdaBoostRegressor__n_estimators': [25, 50, 75]
        }
    elif tecnica == 'HuberRegressor':
        parametros = {
            'HuberRegressor__max_iter': [50, 100, 150],
            'HuberRegressor__warm_start': [True, False]
        }
    elif tecnica == 'PassiveAggressiveRegressor':
        parametros = {
            'PassiveAggressiveRegressor__max_iter': [500, 1000, 1500],
            'PassiveAggressiveRegressor__fit_intercept': [True, False],
            'PassiveAggressiveRegressor__early_stopping': [True, False],
            'PassiveAggressiveRegressor__shuffle': [True, False],
            'PassiveAggressiveRegressor__warm_start': [True, False]
        }
    elif tecnica == 'SGDRegressor':
        parametros = {
            'SGDRegressor__loss': ('squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'),
            'SGDRegressor__penalty': ('l1', 'l2', 'elasticnet', None),
            'SGDRegressor__max_iter': [500, 1000, 1500],
            'SGDRegressor__shuffle': [True, False],
            'SGDRegressor__warm_start': [True, False],
            'SGDRegressor__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
            'SGDRegressor__early_stopping': [True, False]
        }
    elif tecnica == 'TheilSenRegressor':
        parametros = {
            'TheilSenRegressor__fit_intercept': [True, False],
            'TheilSenRegressor__max_iter': [150, 300, 450]
        }
    elif tecnica == 'KNeighborsRegressor':
        parametros = {
            'KNeighborsRegressor__n_neighbors': [3,5,7,9],
            'KNeighborsRegressor__weights': ('uniform', 'distance'),
            'KNeighborsRegressor__algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
            'KNeighborsRegressor__p': [1,2]
        }
    elif tecnica == 'RadiusNeighborsRegressor':
        parametros = {
            'RadiusNeighborsRegressor__weights': ('uniform', 'distance'),
            'RadiusNeighborsRegressor__algorithm': ('ball_tree', 'kd_tree', 'brute', 'auto'),
            'RadiusNeighborsRegressor__p': [1, 2]
        }
    elif tecnica == 'MLPRegressor':
        parametros = {
            'MLPRegressor__hidden_layer_sizes': [(1,), (100,), (500,), (1,3), (100,3), (500,3)],
            'MLPRegressor__activation': ('identity', 'logistic', 'tanh', 'relu'),
            'MLPRegressor__solver': ('lbfgs', 'sgd', 'adam'),
            'MLPRegressor__learning_rate': ('constant', 'invscaling', 'adaptive'),
            'MLPRegressor__shuffle': [True, False],
            'MLPRegressor__max_iter': [100, 200, 300, 1000],
            'MLPRegressor__tol': [0.001, 0.0001, 0.00001]
        }
    elif tecnica == 'DecisionTreeRegressor':
        parametros = {
            'DecisionTreeRegressor__criterion': ('gini', 'entropy'),
            'DecisionTreeRegressor__splitter': ('best', 'random'),
            'DecisionTreeRegressor__max_features': ('auto', 'sqrt', 'log2'),
        }
    elif tecnica == 'ExtraTreeRegressor':
        parametros = {
            'ExtraTreeRegressor__criterion': ('mse', 'mae'),
            'ExtraTreeRegressor__max_features': ('auto', 'sqrt', 'log2')
        }
    return parametros

In [11]:
def tratarPredict(predict):
    """
    Função para tratar os valores preditos.
    É necessário tratar os valores preditos pelos seguintes motivos:
        - O atributo classe 'quality' é do tipo inteiro e deve conter valores entre 0 e 10
        - Uma técnica de regressão pode predizer valores que não são inteiros. Exemplo: 4.27
        - Uma técnica de regressão pode predizer valores abaixo de 0 (valores negativos) e valores acima de 10
        
    Tratamentos realizados:
        - Arredondar valor para número inteiro
        - Se o valor for menor que 0, atribuir o valor de 0
        - Se o valor for maior que 10, atribuir o valor de 10
    """
    
    predict = predict.round()
    predict = predict.astype('int')
    
    predict[predict < 0] = 0
    
    predict[predict > 10] = 10
    
    return predict
    

In [12]:
def salvarDesempenho(resultados):
    """
    Função que recebe um Pandas DataFrame por parâmetro, e salva os dados no arquivo CSV.
    Caso já exista um arquivo, os dados recebidos por parâmetro são adicionados no arquivo
    """
    
    nome_arquivo = 'resultados_treino.csv'    
    
    try:
        csv_resultados = pd.read_csv(nome_arquivo, sep=';') # Abrir arquivo existente
        csv_resultados = pd.concat([csv_resultados, resultados]) # Adicionar dados recebidos por parâmetro ao conjunto de dados já existente
        csv_resultados.to_csv(nome_arquivo, sep=';', index=False) # Salvar os dados 
    except FileNotFoundError: # Caso não exista o arquivo, será criado um novo com os dados recebido por parâmetro
        resultados.to_csv(nome_arquivo, sep=';', index=False)

In [13]:
def avaliarDesempenho(tecnica, y_true, y_pred):
    """
    Função que recebe o valor verdadeiro/real (y_true) e o valor predito (y_pred), 
    aplica os avaliadores de desempenho e retorna o resultado dos avaliadores de desempenho.
    Avaliadores de desempenho aplicados:
        - mean_absolute_error
        - mean_squared_error
        - mean_squared_log_error
        - median_absolute_error
    """
    
    resultados = {}
    resultados['tecnica'] = [tecnica]
    # Aplicando avaliadores de desempenho        
    resultados['mean_absolute_error'] = [sklearn.metrics.mean_absolute_error(y_true, y_pred)]
    resultados['mean_squared_error'] = [sklearn.metrics.mean_squared_error(y_true, y_pred)]
    resultados['mean_squared_log_error'] = [sklearn.metrics.mean_squared_log_error(y_true, y_pred)]
    resultados['median_absolute_error'] = [sklearn.metrics.median_absolute_error(y_true, y_pred)]
        
    log('Técnica: {0} - mean_absolute_error: {1}, mean_squared_error: {2}, mean_squared_log_error: {3}, median_absolute_error: {4}'.format(
            tecnica, resultados['mean_absolute_error'], resultados['mean_squared_error'], resultados['mean_squared_log_error'], resultados['median_absolute_error']) )
    
    pd_resultados = pd.DataFrame(data=resultados)   
    
    return pd_resultados    

In [16]:
def autoModeling(X, y, quantidade_execucoes):
    """
    Função que executa todas as etapas necessárias para separar os dados em treino e teste, e montar o pipeline de treinamento
    O pipeline é composto de:
        - Normalizador e seus parâmetros, se existente
        - Redutor de dimensionalidade e seus parâmetros, se existente
        - Técnica de predição e seus parâmetros
    Após o pipeline estar montado, é realizado uma busca aleatória (Random Search) para encontrar a melhor combinação de parâmetros, essa busca aleatória é executada por 4 vezes
    Após feito a busca aleatória pela melhor combinação de parâmetros, é realizado a predição do 'X_test', para posteriormente aplicar a avaliação de desempenho do modelo de predição
    Aplicado a avaliação de desempenho, seus resultados são salvos em um arquivo .csv
    Posteriomente, o modelo gerado é salvo (.pkl)
    """

    for i in range(1,quantidade_execucoes+1):
    
        log('========== Iniciando execução número: ' + str(i))

        X_train, X_test, y_train, y_test, tipo_split, random_state_split = splitDados(X, y) # Separando os dados em treino e teste


        pipeline = []
        parametros = {}

        normalizador_pipeline, normalizador_parametros = getNormalizador() # Coletando o normalizador, que será definido aleatoriamente
        normalizador_nome = 'Não utilizado'
        if len(normalizador_pipeline) > 0: # Se for definido que terá normalizador, o normalizador é adicionado no pipeline e no array que salva os parâmetros
            pipeline += [normalizador_pipeline]
            parametros.update(normalizador_parametros)
            normalizador_nome = normalizador_pipeline[0]

        redutor_dimensionalidade_pipeline, redutor_dimensionalidade_parametros = getRedutorDimensionalidade() # Coletando o redutor de dimensionalidade, que será definido aleatoriamente
        redutor_dimensionalidade_nome = 'Não utilizado'
        if len(redutor_dimensionalidade_pipeline) > 0: # Se for definido que terá redutor de dimensionalidade, o redutor de dimensionalidade é adicionado no pipeline e no array que salva os parâmetros
            pipeline += [redutor_dimensionalidade_pipeline]
            parametros.update(redutor_dimensionalidade_parametros)
            redutor_dimensionalidade_nome = redutor_dimensionalidade_pipeline[0]


        log('========== Iniciando treinamento com normalizador: {0}, redutor_dimensionalidade: {1}'.format(normalizador_nome, redutor_dimensionalidade_nome))
        

        for tecnica, model in tecnicas_regressao: # Para cada técnica presente na variável "tecnicas_regressao", coletar os parametros da técnica, realizar o random search e avaliar o seu desempenho
            log('Técnica: ' + tecnica)
            try:

                pipeline_tecnica = pipeline.copy()
                pipeline_tecnica += [(tecnica, model)] # Adicionando a técnica no pipeline

                parametros_tecnica = parametros.copy()
                parametros_tecnica.update(getParametrosTecnica(tecnica)) # Coletando os parâmetros da técnica e adicionando no array de parâmetros

                pipeline_final = Pipeline(pipeline_tecnica)

                cross_validation = getNumeroAleatorio('cross_validation') # Coletando um número aleatório para representar o cross_validation do Random Search
                random_state = getNumeroAleatorio('random_state') # Coletando um número aleatório para representar o random_state do Random Search

                # Executando o Random Search com 4 iterações
                modelo = RandomizedSearchCV(n_iter=4, estimator=pipeline_final, param_distributions=parametros_tecnica, cv=cross_validation, random_state=random_state, n_jobs= -1)

                modelo.fit(X_train, y_train)

                y_pred = modelo.predict(X_test) # Realizando a predição para os dados de teste

                y_pred = tratarPredict(y_pred) # Tratar os valores preditos
                
                desempenho_tecnica = avaliarDesempenho(tecnica=tecnica, y_true=y_test, y_pred=y_pred) # Chamando a função que avalia o desempenho da técnica

                # Adicionando informações relevantes e informações para identificar como é o modelo gerado, permitindo que seja reproduzido manualmente caso necessário
                identificador_aleatorio = getNumeroAleatorio('simples', 9999999) # Gerar um identificador aleatório para o resultado dessa técnica
                desempenho_tecnica['identificador_aleatorio'] = [identificador_aleatorio]
                desempenho_tecnica['tipo_split'] = [tipo_split]
                desempenho_tecnica['normalizador'] = [normalizador_nome]
                desempenho_tecnica['redutor_dimensionalidade'] = [redutor_dimensionalidade_nome]
                desempenho_tecnica['pipeline'] = [str(pipeline_tecnica)]
                desempenho_tecnica['parametros'] = [str(parametros_tecnica)]
                desempenho_tecnica['random_state_split'] = [random_state_split]
                desempenho_tecnica['random_state_random_search'] = [random_state]
                desempenho_tecnica['cross_validation_random_search'] = [cross_validation]
                desempenho_tecnica['best_estimator'] = [modelo.best_estimator_]
                desempenho_tecnica['best_params'] = [modelo.best_params_]

                salvarDesempenho(desempenho_tecnica) # Salvando o desempenho da técnica



                nome_arquivo = 'modelos/' + str(identificador_aleatorio) + '_' + tecnica + '.pkl' # Criando o nome do arquivo, ex.: 112302_KNeighborsRegressor.pkl
                joblib.dump(modelo.best_estimator_, nome_arquivo) # Salvando o modelo

            except Exception as e:

                log('Erro durante execução da técnica: ' + tecnica)
                #print(e)
                pass # Passar para a próxima técnica



            del(pipeline_tecnica) # Liberar memória
            del(parametros_tecnica) # Liberar memória

        log('========== Finalizado execução número: ' + str(i))


Ao total foram feitas 20 execuções 

In [18]:
# Executando por 5 vezes o treinamento das técnicas com os dados de treino (X e y) 
autoModeling(X, y, quantidade_execucoes=5)

2019-09-01 16:35:52.103518: ========== Iniciando execução número: 1
2019-09-01 16:35:52.107515: ========== Iniciando treinamento com normalizador: Não utilizado, redutor_dimensionalidade: Não utilizado
2019-09-01 16:35:52.107515: Técnica: RandomForestRegressor
2019-09-01 16:36:15.597991: Técnica: RandomForestRegressor - mean_absolute_error: [0.4270967741935484], mean_squared_error: [0.5225806451612903], mean_squared_log_error: [0.011916322705044688], median_absolute_error: [0.0]
2019-09-01 16:36:15.650959: Técnica: ExtraTreesRegressor
2019-09-01 16:36:44.693914: Técnica: ExtraTreesRegressor - mean_absolute_error: [0.42516129032258065], mean_squared_error: [0.5051612903225806], mean_squared_log_error: [0.011606029125889957], median_absolute_error: [0.0]
2019-09-01 16:36:44.752876: Técnica: BaggingRegressor
2019-09-01 16:36:46.038078: Técnica: BaggingRegressor - mean_absolute_error: [0.41935483870967744], mean_squared_error: [0.4993548387096774], mean_squared_log_error: [0.01125488861488

****
# Próxima etapa: "Evaluation"